# Large Dataset Experiment

## SUSY

In [1]:
from utils.dataloader import DataLoader

df = DataLoader('higgs').df
df.describe()

,target,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
count,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06,5.000000e+06
mean,4.575654e-01,1.000318e+00,2.192580e-05,-4.994657e-05,9.994304e-01,-3.713140e-05,-1.972243e-05,9.999744e-01,3.542352e-05,1.001437e+00,-4.878472e-05,1.000360e+00,9.999559e-01,9.999164e-01,1.000422e+00,1.000112e+00,1.000192e+00,9.994852e-01,2.249139e-01
std,4.981961e-01,6.873341e-01,1.003107e+00,1.001670e+00,6.542200e-01,1.002839e+00,1.001631e+00,8.728877e-01,1.001654e+00,8.901668e-01,1.001594e+00,6.288446e-01,5.841139e-01,4.708406e-01,8.592491e-01,6.207165e-01,6.237672e-01,4.360948e-01,1.969804e-01
min,0.000000e+00,2.548815e-01,-2.102927e+00,-1.734789e+00,4.285860e-01,-2.059306e+00,-1.734202e+00,2.598711e-04,-1.727117e+00,7.693475e-08,-1.671863e+01,2.673070e-01,1.041228e-03,2.048078e-03,0.000000e+00,2.734135e-02,4.452858e-03,3.211849e-07,4.172130e-08
25%,0.000000e+00,5.624837e-01,-7.573512e-01,-8.673567e-01,5.969881e-01,-7.695916e-01,-8.683741e-01,4.784546e-01,-8.661510e-01,3.691849e-01,-4.923983e-01,5.883287e-01,6.223411e-01,6.505438e-01,1.705215e-01,5.985209e-01,5.134394e-01,6.879128e-01,6.908188e-02
50%,0.000000e+00,7.915511e-01,1.342127e-04,-3.880155e-04,7.997993e-01,-3.793710e-04,2.006134e-04,7.738360e-01,-9.172642e-03,8.019273e-01,-8.037898e-02,8.286114e-01,8.781311e-01,9.344703e-01,9.018388e-01,8.355336e-01,9.142514e-01,1.094396e+00,1.672500e-01
75%,1.000000e+00,1.204413e+00,7.576486e-01,8.670981e-01,1.162353e+00,7.692296e-01,8.674778e-01,1.207444e+00,8.687620e-01,1.375392e+00,3.489009e-01,1.211083e+00,1.220313e+00,1.283522e+00,1.613231e+00,1.207994e+00,1.384583e+00,1.369183e+00,3.303652e-01
max,1.000000e+00,2.055345e+01,2.101605e+00,1.734839e+00,3.303562e+01,2.059721e+00,1.734686e+00,2.106888e+01,1.740689e+00,2.338644e+01,2.048790e+01,2.107572e+01,1.616682e+01,6.731210e+00,2.068624e+01,2.115226e+01,1.561370e+01,1.591660e+00,1.000000e+00


In [ ]:
from utils.dataloader import DataLoader

df = DataLoader('susy').df
df.describe()

In [ ]:
from run_svm import run_c_svm as run_svm
run_svm({
    'seed': 0,
    'dataset': 'susy',
    'e0': 0.1, 'e1': 0.3,
    'C1': 1.0,
    'test_size': 0.15,
    'val_size': 0.15,
    'verbose': True,
    'normalize': True,
    'equalize_prior': False
})

In [3]:
import datetime
from utils.misc import set_global_seeds
from utils.dataloader import DataLoader
from utils import logger
from models.nn import MLP, BinaryClassifier, PeerBinaryClassifier, SurrogateBinaryClassifier

def run_nn(args):
    logger.configure(f'logs/nn/{datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")}')
    logger.info(args)
    
    set_global_seeds(args['seed'])

    dataset = DataLoader(args['dataset'])
    X_train, X_test, X_val, y_train, y_test, y_val = dataset.prepare_train_test_val(args)
    classifier = BinaryClassifier(
        model=MLP(feature_dim=X_train.shape[-1], hidsizes=args['hidsize'], dropout=args['dropout']),
        learning_rate=args['lr'],
        loss_func=args['loss'],
    )
    results = classifier.fit(
        X_train, y_train, X_test, y_test,
        batchsize=args['batchsize'],
        episodes=args['episodes'],
        logger=logger,
    )
    return results

In [ ]:
results = run_nn({
    'dataset': 'susy', 
    'e0': 0, 'e1': 0,
    'seed': 0,
    'episodes': 100000,
    'batchsize': 256,
    'hidsize': [32, 32, 32, 32],
    'lr': 0.0007,
    'test_size': 0.3,
    'val_size': 0.3,
    'dropout': 0, 
    'normalize': True,
    'equalize_prior': True,
    'alpha': 1.,
    'loss': 'bce',
    'activation': 'relu',
    'verbose': True,
})

Logging to logs/nn/19-08-27-06-23-29
{'dataset': 'susy', 'e0': 0, 'e1': 0, 'seed': 0, 'episodes': 100000, 'batchsize': 256, 'hidsize': [32, 32, 32, 32], 'lr': 0.0007, 'test_size': 0.3, 'val_size': 0.3, 'dropout': 0, 'normalize': True, 'equalize_prior': True, 'alpha': 1.0, 'loss': 'bce', 'activation': 'relu', 'verbose': True}
------------------------
| ep        | 0        |
| loss      | 0.694    |
| train_acc | 0.5      |
| val_acc   | 0.499    |
------------------------
------------------------
| ep        | 100      |
| loss      | 0.61     |
| train_acc | 0.686    |
| val_acc   | 0.687    |
------------------------
------------------------
| ep        | 200      |
| loss      | 0.47     |
| train_acc | 0.774    |
| val_acc   | 0.775    |
------------------------
------------------------
| ep        | 300      |
| loss      | 0.462    |
| train_acc | 0.783    |
| val_acc   | 0.783    |
------------------------
------------------------
| ep        | 400      |
| loss      | 0.454    

In [ ]:
results = run_nn({
    'dataset': 'susy', 
    'e0': 0, 'e1': 0,
    'seed': 0,
    'episodes': 100000,
    'batchsize': 256,
    'hidsize': [32, 32, 32, 32],
    'lr': 0.001,
    'test_size': 0.3,
    'val_size': 0.3,
    'dropout': 0, 
    'normalize': True,
    'equalize_prior': True,
    'alpha': 1.,
    'loss': 'bce',
    'activation': 'relu',
    'verbose': True,
})

Logging to logs/nn/19-08-27-20-47-11
{'dataset': 'susy', 'e0': 0, 'e1': 0, 'seed': 0, 'episodes': 100000, 'batchsize': 256, 'hidsize': [32, 32, 32, 32], 'lr': 0.001, 'test_size': 0.3, 'val_size': 0.3, 'dropout': 0, 'normalize': True, 'equalize_prior': True, 'alpha': 1.0, 'loss': 'bce', 'activation': 'relu', 'verbose': True}
------------------------
| ep        | 0        |
| loss      | 0.694    |
| train_acc | 0.5      |
| val_acc   | 0.499    |
------------------------
------------------------
| ep        | 100      |
| loss      | 0.58     |
| train_acc | 0.708    |
| val_acc   | 0.709    |
------------------------
------------------------
| ep        | 200      |
| loss      | 0.459    |
| train_acc | 0.779    |
| val_acc   | 0.779    |
------------------------
------------------------
| ep        | 300      |
| loss      | 0.457    |
| train_acc | 0.785    |
| val_acc   | 0.786    |
------------------------
------------------------
| ep        | 400      |
| loss      | 0.45     |

In [ ]:
results = run_nn({
    'dataset': 'susy', 
    'e0': 0, 'e1': 0,
    'seed': 0,
    'episodes': 100000,
    'batchsize': 256,
    'hidsize': [32, 32, 32, 32],
    'lr': 0.001,
    'test_size': 0.15,
    'val_size': 0.1,
    'dropout': 0, 
    'normalize': True,
    'equalize_prior': True,
    'alpha': 1.,
    'loss': 'bce',
    'activation': 'relu',
    'verbose': True,
})

Logging to logs/nn/19-08-27-21-50-32
{'dataset': 'susy', 'e0': 0, 'e1': 0, 'seed': 0, 'episodes': 100000, 'batchsize': 256, 'hidsize': [32, 32, 32, 32], 'lr': 0.001, 'test_size': 0.15, 'val_size': 0.1, 'dropout': 0, 'normalize': True, 'equalize_prior': True, 'alpha': 1.0, 'loss': 'bce', 'activation': 'relu', 'verbose': True}
------------------------
| ep        | 0        |
| loss      | 0.693    |
| train_acc | 0.5      |
| val_acc   | 0.499    |
------------------------
------------------------
| ep        | 1200     |
| loss      | 0.443    |
| train_acc | 0.793    |
| val_acc   | 0.792    |
------------------------
------------------------
| ep        | 1300     |
| loss      | 0.442    |
| train_acc | 0.793    |
| val_acc   | 0.793    |
------------------------
------------------------
| ep        | 1400     |
| loss      | 0.435    |
| train_acc | 0.794    |
| val_acc   | 0.793    |
------------------------
------------------------
| ep        | 1500     |
| loss      | 0.442    

In [ ]:
results = run_nn({
    'dataset': 'susy', 
    'e0': 0, 'e1': 0,
    'seed': 0,
    'episodes': 100000,
    'batchsize': 256,
    'hidsize': [32, 32, 32, 32, 32],
    'lr': 0.001,
    'test_size': 0.3,
    'val_size': 0.3,
    'dropout': 0, 
    'normalize': True,
    'equalize_prior': True,
    'alpha': 1.,
    'loss': 'bce',
    'activation': 'relu',
    'verbose': True,
})

Logging to logs/nn/19-08-27-23-14-28
{'dataset': 'susy', 'e0': 0, 'e1': 0, 'seed': 0, 'episodes': 100000, 'batchsize': 256, 'hidsize': [32, 32, 32, 32, 32], 'lr': 0.001, 'test_size': 0.3, 'val_size': 0.3, 'dropout': 0, 'normalize': True, 'equalize_prior': True, 'alpha': 1.0, 'loss': 'bce', 'activation': 'relu', 'verbose': True}
------------------------
| ep        | 0        |
| loss      | 0.692    |
| train_acc | 0.5      |
| val_acc   | 0.501    |
------------------------
------------------------
| ep        | 100      |
| loss      | 0.588    |
| train_acc | 0.702    |
| val_acc   | 0.703    |
------------------------
------------------------
| ep        | 200      |
| loss      | 0.46     |
| train_acc | 0.779    |
| val_acc   | 0.78     |
------------------------
------------------------
| ep        | 300      |
| loss      | 0.458    |
| train_acc | 0.785    |
| val_acc   | 0.785    |
------------------------
------------------------
| ep        | 400      |
| loss      | 0.451 

------------------------
| ep        | 5300     |
| loss      | 0.428    |
| train_acc | 0.796    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5400     |
| loss      | 0.433    |
| train_acc | 0.796    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5500     |
| loss      | 0.434    |
| train_acc | 0.795    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5600     |
| loss      | 0.438    |
| train_acc | 0.796    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5700     |
| loss      | 0.437    |
| train_acc | 0.795    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5800     |
| loss      | 0.443    |
| train_acc | 0.795    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 5900     |
| loss      | 0.439    |
| train_acc | 0.796    |


------------------------
| ep        | 10800    |
| loss      | 0.437    |
| train_acc | 0.796    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 10900    |
| loss      | 0.436    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 11000    |
| loss      | 0.425    |
| train_acc | 0.796    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 11100    |
| loss      | 0.432    |
| train_acc | 0.796    |
| val_acc   | 0.796    |
------------------------
------------------------
| ep        | 11200    |
| loss      | 0.436    |
| train_acc | 0.796    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 11300    |
| loss      | 0.436    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 11400    |
| loss      | 0.435    |
| train_acc | 0.797    |


------------------------
| ep        | 16300    |
| loss      | 0.428    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16400    |
| loss      | 0.43     |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16500    |
| loss      | 0.437    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16600    |
| loss      | 0.436    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16700    |
| loss      | 0.436    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16800    |
| loss      | 0.43     |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 16900    |
| loss      | 0.431    |
| train_acc | 0.797    |


------------------------
| ep        | 21800    |
| loss      | 0.434    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 21900    |
| loss      | 0.431    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 22000    |
| loss      | 0.434    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 22100    |
| loss      | 0.431    |
| train_acc | 0.797    |
| val_acc   | 0.798    |
------------------------
------------------------
| ep        | 22200    |
| loss      | 0.427    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 22300    |
| loss      | 0.434    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 22400    |
| loss      | 0.428    |
| train_acc | 0.797    |


------------------------
| ep        | 27300    |
| loss      | 0.427    |
| train_acc | 0.797    |
| val_acc   | 0.798    |
------------------------
------------------------
| ep        | 27400    |
| loss      | 0.437    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 27500    |
| loss      | 0.429    |
| train_acc | 0.797    |
| val_acc   | 0.798    |
------------------------
------------------------
| ep        | 27600    |
| loss      | 0.432    |
| train_acc | 0.798    |
| val_acc   | 0.798    |
------------------------
------------------------
| ep        | 27700    |
| loss      | 0.433    |
| train_acc | 0.797    |
| val_acc   | 0.797    |
------------------------
------------------------
| ep        | 27800    |
| loss      | 0.434    |
| train_acc | 0.798    |
| val_acc   | 0.798    |
------------------------
------------------------
| ep        | 27900    |
| loss      | 0.431    |
| train_acc | 0.797    |
